In [1]:
import numpy as np


SAMPLE_SIZE = 10_000
np.random.seed(2025)  # Return same sample every time

# random_noise = np.random.normal(0, 0.1, SAMPLE_SIZE)

x1 = np.random.uniform(0, 1, SAMPLE_SIZE)
x2 = np.random.uniform(0, 1, SAMPLE_SIZE)
y = x1**2 + np.exp(np.sin(x2))

In [2]:
FEATURES = 2
HIDDEN_LAYERS_SIZES = [2]
OUPUTS = 1

xh_weights = np.ones(shape=(HIDDEN_LAYERS_SIZES[0], FEATURES)).T
xh_biases = np.zeros(shape=(HIDDEN_LAYERS_SIZES[0]))

hy_weights = np.ones(shape=(OUPUTS, HIDDEN_LAYERS_SIZES[-1])).T
hy_biases = np.zeros(shape=(OUPUTS))

In [3]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def feed_forward(x, xh_weights, xh_biases, hy_weights, hy_biases):
    h_star = np.matmul(x, xh_weights) + xh_biases
    h = sigmoid(h_star)

    y_star = np.matmul(h, hy_weights) + hy_biases
    y = sigmoid(y_star)

    return y

X = np.matrix([x1, x2]).T.reshape((SAMPLE_SIZE, FEATURES))
y_hat = feed_forward(X, xh_weights, xh_biases, hy_weights, hy_biases)
y_hat

matrix([[0.76068303],
        [0.84505977],
        [0.82556643],
        ...,
        [0.76045346],
        [0.8197518 ],
        [0.81297112]], shape=(10000, 1))